# <span style="color:#C70039">**Módulo de entrenamiento, _trainingModule.py_**</span>

La siguiente notebook muestra el script _trainingModule.py_ versión **SCT-01-RevA**.

### Función principal del módulo de entrenamiento

El módulo de entrenamiento sirve para adquirir y registrar señales de EEG en un formato _.mat_.

- armar nuestra base de datos de señales de EEG.
- entrenar clasificadores y testear los mismos antes de utilizarlos en tiempo real.

### Funcionalidades

- Comunicación con OpenBCI -Synthetic Board, Cyton Board y Ganglion Board-.
- Comunicación serie con Arduino.
- Almacenamiento de señales de EEG adquiridas durante la sesión de entrenamiento en un archivo _.mat_.

##### ¿Por qué es importante el registro de señales de EEG?

Los datos registrados durante las sesiones de entrenamiento serán utilizados para,
- chequear la presencia de SSVEPS,
- probar nuestro módulo de procesamiento y clasificación

<span style="color:#E74C3C">**Importante:**</span> Para las sesiones de entrenamiento será necesario utilizar un **protocólo de adquisición y registro**. ¿Por qué? Porque de esta manera nos aseguramos de que el registro de datos de EEG sea el mismo para todas las personas y para los equipos. El protocolo será provisto por el Docente Director -LB-.


## Revisando script

Importamos algunas librearias

In [ ]:
"""
Created on Wed Jun 23 09:57:43 2021

@author: Lucas Baldezzari

Módulo de control utilizado para adquirir y almacenar datos de EEG.

Los procesos principales son:
    - Seteo de parámetros y conexión con placa OpenBCI (Synthetic, Cyton o Ganglion)
    para adquirir datos en tiempo real.
    - Comunicación con placa Arduino para control de estímulos.
    - Adquisición de señales de EEG a partir de la placa OpenBCI.
    - Control de trials: Pasado ntrials se finaliza la sesión.
    - Registro de EEG: Finalizada la sesión se guardan los datos con saveData() de fileAdmin

"""

import os
import argparse
import time
import logging
import numpy as np
import threading
# import matplotlib.pyplot as plt

# import pyqtgraph as pg
# from pyqtgraph.Qt import QtGui, QtCore

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowError
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, WindowFunctions, DetrendOperations
from ArduinoCommunication import ArduinoCommunication as AC

from DataThread import DataThread as DT
# from GraphModule import GraphModule as Graph       
import fileAdmin as fa

### Función main()

**Nota:** La función _main()_ es llamada cuando ejecutamos el script _trainingModule.py_.

Iremos viendo las diferentes partes de esta función.

#### Comunicación con OpenBCI

Lo primero que hacemos dentro de la función _main()_ es definir los parámetros para comunicarnos con la placa OpenBCI.

Para la comunicación con la placa OpenBCI utilizaremos la libreria **Brainflow**.

- [Sitio oficial de BrainFlow](https://brainflow.org/)
- [Documentación oficial](https://brainflow.readthedocs.io/en/stable/)

Esta libreria nos permitirá adquirir la señal de EEG desde algunas de las placas de OpenBCI -Synthetic Board, Cyton Board y Ganglion Board- en tiempo real.

In [ ]:
def main():
    
    """INICIO DE CARGA DE PARÁMETROS PARA PLACA OPENBCI"""
    """Primeramente seteamos los datos necesarios para configurar la OpenBCI"""
    #First we need to load the Board using BrainFlow
   
    BoardShim.enable_dev_board_logger()
    logging.basicConfig(level=logging.DEBUG)

    parser = argparse.ArgumentParser()
    # use docs to check which parameters are required for specific board, e.g. for Cyton - set serial port
    parser.add_argument('--timeout', type=int, help='timeout for device discovery or connection', required=False,
                        default=0)
    parser.add_argument('--ip-port', type=int, help='ip port', required=False, default=0)
    parser.add_argument('--ip-protocol', type=int, help='ip protocol, check IpProtocolType enum', required=False,
                        default=0)
    parser.add_argument('--ip-address', type=str, help='ip address', required=False, default='')

    #IMPORTENTE: Chequear en que puerto esta conectada la OpenBCI. En este ejemplo esta en el COM4    
    # parser.add_argument('--serial-port', type=str, help='serial port', required=False, default='COM4')
    parser.add_argument('--serial-port', type=str, help='serial port', required=False, default='')
    parser.add_argument('--mac-address', type=str, help='mac address', required=False, default='')
    parser.add_argument('--other-info', type=str, help='other info', required=False, default='')
    parser.add_argument('--streamer-params', type=str, help='streamer params', required=False, default='')
    parser.add_argument('--serial-number', type=str, help='serial number', required=False, default='')
    parser.add_argument('--board-id', type=int, help='board id, check docs to get a list of supported boards',
                        required=False, default=BoardIds.SYNTHETIC_BOARD)
    # parser.add_argument('--board-id', type=int, help='board id, check docs to get a list of supported boards',
    #                     required=False, default=BoardIds.CYTON_BOARD)
    parser.add_argument('--file', type=str, help='file', required=False, default='')
    args = parser.parse_args()

    params = BrainFlowInputParams()
    params.ip_port = args.ip_port
    params.serial_port = args.serial_port
    params.mac_address = args.mac_address
    params.other_info = args.other_info
    params.serial_number = args.serial_number
    params.ip_address = args.ip_address
    params.ip_protocol = args.ip_protocol
    params.timeout = args.timeout
    params.file = args.file
    
    """FIN DE CARGA DE PARÁMETROS PARA PLACA OPENBCI"""

    board_shim = BoardShim(args.board_id, params) #genero un objeto para control de placas de Brainflow
    board_shim.prepare_session()
    time.sleep(2) #esperamos 2 segundos
    
    board_shim.start_stream(450000, args.streamer_params) #iniciamos OpenBCI. Ahora estamos recibiendo datos.
    time.sleep(4) #esperamos 4 segundos

Una vez cargados los parámetros necesarios para establecer una comunicación con la placa OpenBCI lo que hacemos es crear un objeto _BoardShim_ el cual nos permitirá extraer datos desde la placa -señales de EEG, información de los acelerómetros, entre otra información importante-.

Esto lo hacemos así,

```python
    board_shim = BoardShim(args.board_id, params) #genero un objeto para control de placas de Brainflow
    board_shim.prepare_session()
    time.sleep(2) #esperamos 2 segundos
```

Una vez nos conectamos a la placa OpenBCI mediante la linea _board_shim.prepare_session()_ lo siguiente que hacemos es iniciar la transmnisión de datos desde la placa. Esto se hace así,

```python
    board_shim.start_stream(450000, args.streamer_params) #iniciamos OpenBCI. Ahora estamos recibiendo datos.
    time.sleep(4) #esperamos 4 segundos
```

<span style="color:#E74C3C">**Importante:**</span> Debemos tener en cuenta que al hacer _board_shim.start_stream(450000, args.streamer_params)_ le estamos indicando al objeto _board_shim_ que vamos a reservar un buffer de _450000_ muestras.

Si por ejemplo consideramos que la frecuencia de muestreo de la Cyton Board es de _250Hz_ entonces con este buffer podríamos almacenar unos 30 minutos de datos.

Sin embargo para el caso que nosotros necesitamos, el buffer puede ser mas pequeño ya que solamente necesitaremos las muestras correspondientes al tiempo que duran los estímulos encendidos.

#### Objeto para extraer datos desde los canales de EEG de la OpenBCI

La clase _DataThread_ [autor LB] nos permite extraer cierta cantidad de **samples** desde la OpenBCI. Esto es importante, ya que luego de cierto tiempo -por ejemplo, el tiempo que transcurre durante la fase de estimulación- vamos a extrar datos para procesarlos y obtener un comando.

La creación de un objeto _DataThread_ lo hacemos facilmente como sigue.

```python
    data_thread = DT(board_shim, args.board_id) #genero un objeto DataThread para extraer datos de la OpenBCI
    time.sleep(1)
```

Notar que uno de los parámetros que le pasamos a la clase es _board_shim_ ya que es desde ahí donde vamos a poder extraer los datos.

#### Control de trials

Lo siguiente que hacemos es genear algunas variables que nos permitirán controlar la cantidad y duración total de los trials, como así también el tiempo que queremos que los estímulos estén encendidos.

```python
"""Defino variables para control de Trials"""
    
    trials = 5 #cantidad de trials. Sirve para la sesión de entrenamiento.
    #IMPORTANTE: trialDuration SIEMPRE debe ser MAYOR a stimuliDuration
    trialDuration = 3 #secs
    stimuliDuration = 2 #secs

    saveData = True
    
    EEGdata = []
    fm = 250
    
    samplePoints = int(fm*stimuliDuration)
    channels = 8
    stimuli = 1 #one stimulus
```

**IMPORTANTE:** _trialDuration_ SIEMPRE debe ser MAYOR a _stimuliDuration_.

#### Control de trials con la clase _ArduinoCommunication_

Una vez que hemos establicido una conexión con la placa OpenBCI el siguiente paso es iniciar una comunicación con el <span style="color:#F37263">**Arduino M1**</span> para así tener un control sobre los estímulos y para poder enviar y recibir información del <span style="color:#008a3e">**Arduino M3**</span> a través del <span style="color:#F37263">**Arduino M1**</span>.

Para establecer una comunicación con el arduino usamos la clase _ArduinoCommunication_ [autor LB] y le pasamos como parámetros el puerto _COM_ donde estaría conectado la placa arduino, la duración -en segundos- de un trial, la duración -en segundos- del tiempo que los estímulos estarán encendidos, la cantidad de trials, entre otros parámetros propios del funcionamiento de la clase.

```python

"""Inicio comunicación con Arduino instanciando un objeto AC (ArduinoCommunication)
    en el COM3, con un timing de 100ms
    
    - El objeto ArduinoCommunication generará una comunicación entre la PC y el Arduino
    una cantidad de veces dada por el parámetro "ntrials". Pasado estos n trials se finaliza la sesión.
    
    - En el caso de querer comunicar la PC y el Arduino por un tiempo indeterminado debe hacerse
    ntrials = None (default)
    """
    #IMPORTANTE: Chequear en qué puerto esta conectado Arduino.
    #En este ejemplo esta conectada en el COM3
    ard = AC('COM3', trialDuration = trialDuration, stimONTime = stimuliDuration,
             timing = 100, ntrials = trials)
    time.sleep(2) 
```


#### Preparando variables para almacenar datos

Dijimos que uno de los objetivos principales de la clase _trainingModule_ es poder almacenar los datos luego de una sesión de entrenamiento.

La variable _dictionary_ es un diccionario de Python que contiene información relevante que nos será de utilidad posteriormente para analizar nuestros datos, veamos cuales son los _keys_ de este diccionario.

- 'subject': Nombre o identificador del sujeto que estará por realizar la sesión de entrenamiento. Es un _string_.
- 'date': Fecha que se realiza la sesión de entrenamiento. Es un _string_.
- 'generalInformation': Información general que se crea pertinente almacenar. Por ejemplo, se podría colocar información acerca de la distancia a la cual se colocaron los estíulos o cualquier otra cosa que se crea relevante. Es un _string_.
- 'stimFrec': Frecuencia del estímulo usado durante la sesión de entrenamiento -en Hertz-. Es un _string_.
- 'channels': Lista con los números de canales o con los nombres de canales utilizados durante la sesión de entrenamiento.
- 'dataShape': Es una lista que contiene la forma en que se guardarán los datos de EEG registrados durante la sesión de entrenamiento, por defecto la forma es [stimuli, channels, samplePoints, trials]
- 'eeg': Señal de EEG adquirida desde la OpenBCI durante la sesión de entrenamiento. **Es importante** tener en cuenta que los datos de 'eeg' deben almacenarse con la forma establecida en 'dataShape'.
                    
```python
    path = "recordedEEG" #directorio donde se almacenan los registros de EEG.
    
    #El siguiente diccionario se usa para guardar información relevante cómo así también los datos de EEG
    #registrados durante la sesión de entrenamiento.
    dictionary = {
                'subject': 'Sujeto1Test1',
                'date': '27/08/2021',
                'generalInformation': 'Estímulo a 30cm. Color rojo',
                'stimFrec': "7",
                'channels': [1,2,3,4,5,6,7,8], 
                 'dataShape': [stimuli, channels, samplePoints, trials],
                  'eeg': None
                    }
```

##### Iniciamos sesión

Para iniciar sesión con el <span style="color:#F37263">**Arduino M1**</span> hacemos,

```python
ard.iniSesion() #Inicio sesión en el Arduino.
```


**Nota:** Recordar que la clase _ArduinoCommunication_ posee métodos que nos permite controlar el inicio y finalización de cada tríal, como asi también nos da información si estamos en la fase de estimulación o de la fase sin estimular.

#### Empezamos a estimular y a registrar datos.

Una vez establecemos la comunicación con la placa OpenBCI y el <span style="color:#F37263">**Arduino M1**</span> estamos en condiciones de empezar a estimular y a registrar la señal de EEG.

Analicemos el siguiente pedazo de código.

```python
try:
        while ard.generalControl() == b"1":
            if saveData and ard.systemControl[1] == b"0":
                currentData = data_thread.getData(stimuliDuration)
                EEGdata.append(currentData)
                saveData = False
            elif saveData == False and ard.systemControl[1] == b"1":
                saveData = True
        
    except BaseException as e:
        logging.warning('Exception', exc_info=True)
        
    finally:
        if board_shim.is_prepared():
            logging.info('Releasing session')
            board_shim.release_session()
            
        #ard.endSesion() #finalizo sesión (se apagan los estímulos)
        ard.close() #cierro comunicación serie para liberar puerto COM
        
        #Guardo los datos registrados por la placa
        EEGdata = np.asarray(EEGdata)
        rawEEG = EEGdata.reshape(1,EEGdata.shape[0],EEGdata.shape[1],EEGdata.shape[2])
        rawEEG = rawEEG.swapaxes(1,2).swapaxes(2,3)
        dictionary["eeg"] = rawEEG
        fa.saveData(path = path,dictionary = dictionary, fileName = dictionary["subject"])
```

Podemos ver que mientras _generalControl()_ sea igual a _b"1"_, ejecutaremos lo que esta dentro del _while_. Recordar que el método _generalControl()_ de la clase _ArduinoComminucation_ nos devuelve el estado de la sesión del Arduino. Mientras la cantidad de trials ejecutados por la clase _ArduinoComminucation_ sea menor o igual a los _trials_ que pasamos como parámetro cuando creamos el objeto _ArduinoComminucation_ el método _generalControl()_ nos devolverá _b"1"_, caso contrario nos devolverá un _b"0"_ indicando que la sesión ha finalizado y por lo tanto saldremos el bucle _while_.

Por otro lado, mientras estemos dentro del bucle _while_ vemos que si se cumple la linea _if saveData and ard.systemControl[1] == b"0"_ entonces hacemos dos cosas, 
1) Extraemos un pedazo de mi señal de EEG desde la OpenBCIE haciendo _data_thread.getData(stimuliDuration)_ con una duración igual a _stimuliDuration_, es decir, igual al tiempo en que los estímulos estuvieron encendidos.

2) Los datos extraidos lo agregamos a la lista de _EEGData_ haciendo _EEGdata.append(currentData)_.

Cuando se ejecutan la cantidad de trials establecidas, la sesión finaliza y por último,
- Cerramos la comunicación con la placa OpenBCI haciendo _board_shim.release_session()_.
- Cerramos la comunicación con la placa <span style="color:#F37263">**Arduino M1**</span> haciendo _ard.close()_.
- Guardamos los datos de la variable _EEGdata_.

Vemos que los datos de EEG almacenados en la variable _EEGdata_ se guardan dentro del diccionario llamado _dictionary_ que creamos anteriormente. Finalmente se utiliza el método _saveData()_ de la librearia _fileAdmin_ [autor LB] para guardar todo en un archivo _'.mat'_.

Si revisamos la carpeta _\recordedEEG_ podremos ver que se ha generado un archivo _'Sujeto1Test1.mat'_.

![Sujeto1Test1.png](Sujeto1Test1.png)

## <span style="color:#C70039">¿Y luego de la sesión de entrenamiento? </span>

Una vez que realizamos la sesión de entrenamiento tendremos datos para realizar diferentes pruebas, entre las más importantes tenemos,

- Chequear la presencia de SSVEPs.
- Entrenar nuestros algorítmos de clasificación para detectar SSVEPs y así determinar un comando para el vehículo robótico.
- Testear nuestros clasificadores.

## <span style="color:#C70039"> ¿Hay algo más? </span>

**¡SI!**

El siguiente paso es implementar un módulo de <span style="color:#2874A6">**procesamiento y clasificación**</span> que nos permita tomar datos de EEG, filtrarlos y clasificarlos **todo en tiempo real** para así enviar un comando al _Módulo 3_ encargado de controlar los movimientos del vehículo. Dicho módulo ya esta en proceso de ser implementado. Esatrá formado por varios de las funciones que ya hemos visto en otros talleres.

<span style="color:#A93226">**¡Vamo' arriba! que estamos cerca.**</span>

![lobo.gif](lobo.gif)